In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import argparse
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from csv import DictWriter
from threading import Thread
import os, csv, re, queue, time, winsound, traceback
import pycountry

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--ignore-ssl-errors')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--incognito')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
caps = webdriver.DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"
service = Service(ChromeDriverManager().install())

LINK = 'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&field1=AllField&text1=LEACH+AND+WSN+AND+ENERGY&pageSize=50&expand=all'

In [7]:
driver = webdriver.Chrome(service=service, options=chrome_options)
link ='https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&field1=AllField&text1=LEACH+AND+WSN+AND+ENERGY&pageSize=50&expand=all'

driver.get(link)
allow_all_cookies_button = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
)

# Click the 'Allow all cookies' button
allow_all_cookies_button.click()
html = driver.page_source
driver.close()
soups = BeautifulSoup(html, 'html.parser')


In [9]:
article = soup.find_all('li', class_='search__item issue-item-container')[1]

driver = webdriver.Chrome(service=service, options=chrome_options)
try:
    link_article        = 'https://dl.acm.org' + article.find('h5', class_='issue-item__title').find('a')['href']
    article_type       = article.find('div', class_='issue-heading').get_text()
    article_title = article.find('h5', class_='issue-item__title').get_text()

    driver.get(link_article)
    driver.implicitly_wait(5)
    allow_all_cookies_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
    )
    allow_all_cookies_button.click()
    html = driver.page_source

    article_soup = BeautifulSoup(html, 'html.parser')
    publication_title = article_soup.find('div', class_='core-enumeration')
    if publication_title != None:
        publication_title = publication_title.get_text()
    else:
        publication_title = article_soup.find('div', property='isPartOf')
        publication_title = publication_title.get_text()
            # if publication_title != None:                   

    info_click = driver.find_element(By.XPATH, '//*[@id="article_collateral_menu"]/ul/li[1]/a')
    driver.execute_script("arguments[0].scrollIntoView(true);", info_click)

    info_click =  WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="article_collateral_menu"]/ul/li[1]/a'))
    )
    info_click.click()
    html = driver.page_source
    article_soup_info = BeautifulSoup(html, 'html.parser')

    contribution_button = driver.find_element(By.ID, 'tab-contributors-label')
    contribution_button.click()

    contribution_button =  WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.ID, "tab-contributors"))
    )
    html = driver.page_source
    article_soup_author = BeautifulSoup(html, 'html.parser')

    number_of_citation = article_soup.find('span', class_="citation")

    if number_of_citation:
        number_of_citation = number_of_citation.get_text().replace('citation', '')
    else:
        number_of_citation = 'none'

    abstract = article_soup.find('section', id="abstract") 

    if abstract != None:
        abstract = abstract.get_text().replace("Abstract", '')
    else:
        abstract = article_soup.find('section', id="author-abstract") 
        abstract = abstract.get_text().replace("Abstract", '')
        
    publication_date = article_soup.find('span', class_='core-date-published').get_text()

    keywords = article_soup_info.find('section', property='keywords')
    if keywords != None:
        keywords = keywords.find_all('li')
        keywords = ','.join([keyword.get_text() for keyword in keywords])
    else:
        keywords = 'none'

    author_informations = article_soup_author.find('div', class_='contributors').find_all('span', property="author")
    authors = ''
    countries = ''
    affiliations = ''
    count = 0

    for author_information in author_informations:
        author = author_information.find('a').get_text()
        affiliation = author_information.find('div', class_='content').get_text()
        affiliation = affiliation.replace('View Profile', '')

        authors += author
        affiliations += f'({affiliation})'

        affiliation = affiliation.split(', ')
        country = affiliation[len(affiliation)-1]
        countries += country

        if count < len(author_informations)-1:
            authors         += ','
            countries       += ','
            affiliations    += ','
        count += 1
    
    occurrences = countries.split(',')
    found_countries = []

    for country in pycountry.countries:
        for occurrence in occurrences:
            if country.name in occurrence:
                found_countries.append(country.name)

    countries = ','.join(found_countries)
        

    
    row_data = {
        'title'                         : article_title,
        'link'                          : link_article,
        'number_of_citation'            : number_of_citation,
        'article_type'                  : article_type,
        'publisher'                     : 'ACM',
        'keyword'                       : keywords,
        'abstract'                      : abstract,
        'publish_date'                  : publication_date,
        'publication_title'             : publication_title,
        'authors'                       : authors,
        'affiliations'                  : affiliations,
        'countries'                     : countries
    }
    
    for col in row_data.keys():
        print(f"{col}:\n\t\t\t\t{row_data[col]}")

except Exception as e:
        print("\n"*3)
        print("== ! Error ! =="*10)
        print("\n"*1)
        print(f"Failed to retrieve article: \n{article_title}\n{link_article}\n{e}")
        traceback.print_exc()
        print("\n"*3)
        print("== ! Error ! =="*10)
        print("\n"*1)

title:
				Energy-Aware MAX-LEACH Routing Protocol for Homogeneous and Heterogeneous WSNs
link:
				https://dl.acm.org/doi/10.1007/s11277-023-10673-0
number_of_citation:
				0
article_type:
				research-article
publisher:
				ACM
keyword:
				Average energy,Sink,Cluster head,Energy consumption,Heterogeneous WSN,LEACH protocol,Network lifetime
abstract:
				The Energy-Efficient wireless sensor network is needed in the present-day Scenario. The main problem in the area of WSN is energy consumption or low battery power. In such a case the best routing protocols are required to reduce the energy consumption and to improve the network lifetime. This paper is focused on how to minimize the nodes' energy depletion and then how to rise the network lifetime by efficient routing protocols in the WSN. In this paper, we have proposed an Energy-Aware Routing protocol for Homogeneous and Heterogeneous WSNs called MAX LEACH. The proposed protocol improves the total network lifetime. The proposed prot